# Lab 3.3: Phát hiện biển báo giao thông

Chào mừng bạn đến với Lab 3.3. Trong bài lab này, chúng ta sẽ học cách sử dụng mô hình YOLO đã được huấn luyện trước để phát hiện phương tiện.

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết để giúp bạn lập trình và chạy chương trình theo dõi phương tiện.


### Thư viện

In [ ]:
import torch
from PIL import Image
import cv2
import numpy as np
import os
import pathlib
# Ensure proper path handling for Windows
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Handle library conflicts
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
class VehicleDetector():
    def __init__(self, model_path, force_reload):
        self.model = self.load_model(model_path, force_reload)
        self.classes = self.model.names
        print(self.classes)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        print("Using Device: ", self.device)

    def load_model(self, model_path, force_reload):
        if model_path:
            model = torch.hub.load(
                'ultralytics/yolov5',
                'custom',
                path=model_path,
                force_reload=force_reload
            )
        else:
            model = torch.hub.load(
                'ultralytics/yolov5',
                'yolov5s',
                pretrained=True
            )

        return model

    def score_frame(self, frame):
        frame_rgb = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        results = self.model(frame_rgb)

        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

        return labels, cord

    def class_to_label(self, x):
        return self.classes[int(x)]

    def plot_boxes(self, results, height, width, confidence=0.3):
        labels, cord = results
        detections = []

        for i in range(len(labels)):
            row = cord[i]
            if row[4] >= confidence:
                x1, y1, x2, y2 = int(
                    row[0]*width), int(row[1]*height), int(row[2]*width), int(row[3]*height)

                conf = float(row[4].item())
                class_label = self.class_to_label(labels[i])
                # print(feature)
                detections.append(
                    ([x1, y1, int(x2-x1), int(y2-y1)], conf, class_label))

        return detections


### Triển khai Mô hình YOLOv5 đã huấn luyện trước và DeepSORT




Đáp án bài tập 1

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
ip_lap_lead = "192.168.109.105"
port_lap_lead = 7001

ip_lap = "192.168.109.105"
port_lap = 3000

# Define the path to the YOLO model weights for traffic sign detection
model_path = r'..\Other\YoloWeights\VehicleDetect.pt'

# Initialize the traffic sign detector with the specified model path
vehicle_detector = VehicleDetector(
    model_path=model_path, force_reload=True)

# Initialize the object tracker with specified parameters
object_tracker = DeepSort(
    max_age=3, n_init=2, nms_max_overlap=1.0, max_cosine_distance=0.3)

In [ ]:
import socket
BUFFER_SIZE = 65355


def create_udp_socket_listen(ip, port):
    """Create and bind a UDP socket listening"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((ip, port))
    return sock


def create_udp_socket_send():
    """Create and bind a UDP socket for sending"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, BUFFER_SIZE)
    return sock

In [ ]:
from queue import Queue

END_MARKER = b'\xff\xd9'
IMAGE_RESIZE_DIMS = (1280, 720)
queue_image = Queue()
MAX_QUEUE_SIZE = 3


def get_image_from_cam():
    """Receive images via UDP and put them in the queue."""
    global stop_event
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while cap.isOpened() and (not stop_event.is_set()):
        _, image = cap.read()
        queue_image.put(image)

        if queue_image.qsize() >= MAX_QUEUE_SIZE:
            queue_image.get()

In [ ]:
def get_image_from_leader():
    """Receive images via UDP and put them in the queue."""
    global stop_event

    sock = create_udp_socket_listen(ip_lap, port_lap)
    print(f"Listening on IP {ip_lap}, UDP port {port_lap}")

    image_data = bytearray()
    while not stop_event.is_set():
        data, _ = sock.recvfrom(BUFFER_SIZE)
        image_data.extend(data)

        if data.endswith(END_MARKER):
            try:
                image_array = np.frombuffer(image_data, dtype=np.uint8)
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                queue_image.put(image)

                if queue_image.qsize() >= MAX_QUEUE_SIZE:
                    queue_image.get()
            except Exception as e:
                print(f"Failed to convert image: {str(e)}")

            image_data = bytearray()

In [ ]:
import time

# Define Direction constants
STRAIGHT = 1
SLOW = 2
TURNLEFT = 5
TURNRIGHT = 6
FORWARD = 7
BACKWARD = 8
STANDBY = 9
STOP = 10


def detect_vehicle():
    prev_time = time.time()
    frame_count = 0
    fps = 0
    prev_cmd = STOP

    sock = create_udp_socket_send()

    while True:
        try:
            image = queue_image.get()

            frame_count += 1
            tmp_time = time.time() - prev_time
            if tmp_time >= 1:
                fps = frame_count / tmp_time
                prev_time = time.time()
                frame_count = 0

            results = vehicle_detector.score_frame(image)
            detections = vehicle_detector.plot_boxes(
                results, height=image.shape[0], width=image.shape[1], confidence=0.5)
            tracks = object_tracker.update_tracks(detections, frame=image)

            cmd = STRAIGHT
            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox_x1, bbox_y1, bbox_x2, bbox_y2 = map(int, track.to_ltrb())
                class_label = track.det_class
                conf = track.det_conf if track.det_conf is not None else 0.0

                if bbox_y2 >= image.shape[0] - 100:
                    cmd = STOP

                    # Change color
                    cv2.rectangle(image, (bbox_x1, bbox_y1),
                                  (bbox_x2, bbox_y2), (0, 255, 0), 2)
                else:
                    # Normal tracking display
                    cv2.rectangle(image, (bbox_x1, bbox_y1),
                                  (bbox_x2, bbox_y2), (0, 0, 255), 2)

                cv2.putText(image, "ID: " + str(track_id), (bbox_x1, bbox_y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(image, f'Class: {
                            class_label}', (bbox_x1, bbox_y1 - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.putText(image, f'Conf: {
                    conf:.2f}', (bbox_x1, bbox_y1 - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        except Exception as e:
            print("Error processing image:", e)
            cmd = STOP

        if cmd != prev_cmd:
            try:
                data_cmd = f"{cmd}"
                data_bytes_cmd = data_cmd.encode()
                sock.sendto(data_bytes_cmd, (ip_lap_lead, port_lap_lead))
                print(f"Sent UDP message: '{cmd}' to {
                      ip_lap_lead}:{port_lap_lead}")
                prev_cmd = cmd
            except Exception as e:
                print(f"Error sending UDP message: {e}")

        cv2.putText(image, f'FPS: {int(fps)}', (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        cv2.imshow('image', image)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    # sock.close()
    cv2.destroyAllWindows()

### Kiểm tra mô hình phát hiện phương tiện bằng cách sử dụng camera laptop



Đáp án bài tập 2

In [ ]:
import threading
stop_event = threading.Event()


threads = [
    threading.Thread(target=get_image_from_cam,
                     name='GetSendImageThread'),
    threading.Thread(target=detect_vehicle,
                     name='ProcessImageThread'),
]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

### Triển khai mô hình phát hiện phương tiện để điều khiển bộ AutoCar



Đáp án bài tập 3

In [ ]:
import threading
stop_event = threading.Event()


threads = [
    threading.Thread(target=get_image_from_leader,
                     name='GetSendImageThread'),
    threading.Thread(target=detect_vehicle,
                     name='ProcessImageThread'),
]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()